<a href="https://colab.research.google.com/github/sethorus30/API_Project/blob/master/Capstone1.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from     bs4 import  BeautifulSoup as soup
import   requests
import   pandas as pd
from     urllib.request  import  urlopen
import   xml.etree.ElementTree  as ET
import   copy

# Create list for Maker’s home page URL
makers_List = ['https://www.kbb.com/acura/' , 'https://www.kbb.com/alfaromeo/' , 'https://www.kbb.com/astonmartin/' , 'https://www.kbb.com/audi/' , 'https://www.kbb.com/bentley/',  'https://www.kbb.com/bmw/' , 'https://www.kbb.com/buick/' , 'https://www.kbb.com/cadillac/' , 'https://www.kbb.com/chevrolet/' , 'https://www.kbb.com/chrysler/' , 'https://www.kbb.com/dodge/' , 'https://www.kbb.com/ferrari/' , 'https://www.kbb.com/fiat/' , 'https://www.kbb.com/ford/' , 'https://www.kbb.com/freightliner/' , 'https://www.kbb.com/genesis/' , 'https://www.kbb.com/gmc/' , 'https://www.kbb.com/honda/' , 'https://www.kbb.com/hyundai/' , 'https://www.kbb.com/infiniti/' , 'https://www.kbb.com/jaguar/' , 'https://www.kbb.com/jeep/' , 'https://www.kbb.com/kia/' , 'https://www.kbb.com/lamborghini/' , 'https://www.kbb.com/landrover/',  'https://www.kbb.com/lexus/' , 'https://www.kbb.com/lincoln/' , 'https://www.kbb.com/maserati/' , 'https://www.kbb.com/mazda/' , 'https://www.kbb.com/mclaren/' , 'https://www.kbb.com/mercedesbenz/' , 'https://www.kbb.com/mini/' , 'https://www.kbb.com/mitsubishi/' , 'https://www.kbb.com/nissan/' , 'https://www.kbb.com/porsche/', 'https://www.kbb.com/ram/' , 'https://www.kbb.com/rivian/' , 'https://www.kbb.com/rollsroyce/' , 'https://www.kbb.com/smart/' , 'https://www.kbb.com/subaru/' , 'https://www.kbb.com/tesla/' , 'https://www.kbb.com/toyota/' , 'https://www.kbb.com/volkswagen/' , 'https://www.kbb.com/volvo/']

# Create list for annual inflation rate in the USA  (2020 - 1964)
# URL: https://www.usinflationcalculator.com/inflation/historical-inflation-rates/  
annual_inflation_rate_List = [0 , 0.018, 0.024, 0.021, 0.013, 0.001, 0.016, 0.015, 0.021, 0.032, 0.016, -0.004, 0.038, 0.028, 0.032, 0.034, 0.027, 0.023, 0.016, 0.028, 0.034, 0.022, 0.016, 0.023, 0.03, 0.028, 0.026, 0.03, 0.042, 0.054, 0.048, 0.041, 0.036, 0.019, 0.036, 0.043, 0.032, 0.062, 0.103, 0.135, 0.113, 0.076, 0.065, 0.058, 0.091, 0.11, 0.062, 0.032, 0.044, 0.057, 0.055, 0.042, 0.031, 0.029, 0.016, 0.013, 0.013]

mListSize = len(makers_List)
print ('Maker List size = ' , mListSize)
print("\n\n")
prop = {}
tmp = {}
tmp_dct_p = {}
tmp_dct = {}
links = []
Maker = ''
first_year = ''
i = 0
m = []
values = []
model_dct_list = []                 # list of all models for a given maker
tmp_dct_list = []       # list of dictionaries structure used by called functions.
model_years_dct_list = []  # holds detailed model info (model, first-year, price, model_url>
model_final_dct_list = []      # final including all years for given model

# input:  maker_url
# output: {Maker : maker, Model : v1,  First Year : v2,  New_Price : v3,  url : link}, {}….] }
def  get_maker_models(maker_url) : 
    r3 = requests.get(maker_url)
    #print("maker_url: ", maker_url)
    soup1 = soup( r3.content, 'html.parser')

    # find iterative top level TAG for each maker/model
    link_as = soup1.find('div', {"class" : "make-list-wrapper"}).find_all('div', {"class" : "results-row js-results-row"})
    
    i = 0
    # y is the top-level TAG for 1st year models
    # from that top level get: links, maker, model, first year for all 
    # top-year Models for that Maker’s new Cars
    for   y  in   link_as :       
        x = y.find('a', href=True)
        link = "https://www.kbb.com" + x['href']   # x['href'] has '/Maker/Model'
        if link not in links:
            links.append(link)
            maker, model = (x['href'].strip('/')).split('/')
            Maker = maker[:]
            # get the first year from "2019 Aston Marton DB11"
            ymm = x.find('h3', {"class" : "title-three"}).string 
            print("ymm: ", ymm)            
            m = ymm.split()
            print("9: ", m[0], type(m[0]))                                            # m[0] is 1st year = 2019 is a string
            # first-year = m[0]
            # get the price for the first year
            price = y.find_next('span', {"class" : "title-four"}).text
            print ("price = ", price)
            L1 = ['Maker', 'Model', 'Year', 'New_Price', 'Model-url']
            L2 = [Maker, model, m[0], price, link]
            dct = dict(zip(L1, L2))
            if  i == 0 :
                # copy dct to model_dct_list[0]
                model_dct_list.insert(0, dct.copy())
            else : 
                model_dct_list.append(dct.copy())
            i = i + 1

    print ("\n\n")   
    #print("1: ", model_dct_list) 
    return    model_dct_list                

# Input: model, fyear, fprice, model_url   1st year data for specific maker/model
# Output: [ {model1, year1, price1, model_url1}, {model1, year2, price2, model_url2}, ….]

def   get_maker_model_years_price (maker, model, f_year, f_price, model_url) :
    cur_price = 0
    next_year_price = 0
    r = requests.get(model_url)
    # create soup2 object for the model page for a specific Maker (to search 
    # recursively for <year and Link> )
    soup2 = soup( r.content, 'html.parser')
    # find years and corresponding links for this specific Maker/Model/First-
    # Year/New_Price/Model_url of new Cars
    i = 0
    # convert price (f_price) from string ($xx,xxx) into integer binary representation
    pp = f_price.replace('$', '')
    pp = pp.replace(',', '')
    cur_price = int(pp)                                                       # convert string pp into integer
    next_year_price = cur_price          # integer binary representation
    
    # write/append first year data for a given model into tmp_dct_list[]
    tmp_dct = {}
    tmp_dct_list.clear()
    L1 = ['Maker', 'Model', 'Year', 'New_Price', 'Model-url']
    L2 = [maker, model, f_year, f_price, model_url]
    tmp_dct = dict(zip(L1, L2))
    # add first year for this specific maker/model
    tmp_dct_list.insert(0, tmp_dct.copy())  
    
    # find index in the annual_inflation_rate_list[] for this model
    if  '2020' in f_year :                  # 2020
        i = 1                               # inflation starts from 2019
    elif  '2019'  in  f_year :              # 2019
        i = 2                               # inflation starts from 2018
    elif   '2018'  in f_year :              # 2018
        i = 3                               # inflation starts from 2017
    else :
        i = -1                               # undefined
        
    last_year = f_year[ : ]
    n = 0                                                                     # skip first year as I have it already
    # remaining years below
    for  y  in  soup2.find_all("li", {"data-analytics" : True}) : 
        if  n == 0 :             # skip first year as I have it already
            n = n + 1
            continue
        mm_year = y.find_next('a', href=True)# mm_year = “/acura/ilx/2019”
        link = "https://www.kbb.com" + mm_year['href']
        year = y.find_next('a', href=True).contents    # year = “2019” 
        print("type(year): ", type(year))
        print("year: ", year)
        yr = str(year[0])                               # string
        print ("f_year, p_year", f_year, yr)
        if yr == last_year or yr == "" :
            i = i + 1
            continue
        last_year = yr[ : ]
        # estimated cur_price (binary) = 
        #              int(next_year_price  / (1 + annual_inflation_rate_List[i]) )
        # cur_price is integer binary representation
        cur_price = int(next_year_price / (1 + annual_inflation_rate_List[i]) ) 
        next_year_price = cur_price
        p1 = str(cur_price)                # convert binary into string
        l = len(p1) - 3                    # l = (string length – 3)
        p2 = p1[:l] + ',' +  p1[l:]
        p = "$" + p2         # p is current price in string with right format
        #p = "N/A"
        if (len(tmp_dct)) :
            tmp_dct.clear()
        # Needs to adjust the URL by appending year
        m_url = model_url + yr
        L1 = ['Maker', 'Model', 'Year', 'New_Price', 'Model-url']
        L2 = [maker, model, year, p, m_url]
        tmp_dct = dict(zip(L1, L2))         
        tmp_dct_list.append (tmp_dct.copy() )
        i = i + 1                                                                  # index into annual inflation rate array
    return  tmp_dct_list     # [ {<maker: <model, year, price, model_url} ]

# input: maker/model/<year : URL> and output: maker/model/years/price/{properties dictionary}
def get_maker_model_year_price_properties(maker, model, year, price, model_year_url) :
    prop.clear()                           # clear the dictionary
    technology = []
    entertainment = []
    #tmp_dct = {}
    i = 0    
    r = requests.get(model_year_url)
    # create soup3 object for the top level HTML page (to search recursively
    # for makers, models and Links)
    soup3 = soup( r.content, 'html.parser')  
    # print(soup3.find('div', {"class" : "css-1wbw7cx epomrkc1"}).find('div', {"class":"stars"}).text)
    # (1-2) find out MPG and HorsePower
    mpg = False
    hp = False
    print("get_maker_model_year_price_properties - year: ", year)

    for p  in soup3.find('div', {"class" : "css-1wbw7cx epomrkc1"}).find_all('div', {"class" : "stars"}):
        parent = str(p.find_parent('div').text).strip()
        #print("parent: ", parent)
        if  "Combined Fuel Economy" in   parent :
            #prop['MPG'] = p.text
            M_P_G = p.text
            mpg = True
        elif  "Horsepower"  in   parent :
            #prop['HP'] = p.text
            H_P = p.text
            hp = True
        else :
            break
    if mpg == False :
        #prop['MPG'] = "N/A"
        M_P_G = "N/A"
    if hp == False :
          #prop['HP'] = "N/A"
          H_P = "N/A"

    # (3-4) find out Expert Rating and Consumer rating
    i = 0
    er = False
    cr = False
    for p in soup3.find_all('div', {"class":"css-x9skgx-NumericRating e149jidm1"}) :
        i = i + 1
        if i == 1 :
            if  p.text :
                value = p.text.replace('\xa0', '')
                #prop['Expert-Rating'] = value
                E_R = value
                er = True
            else :
                prop['Expert-Rating'] = "N/A"
                E_R = "N/A"
        elif (i == 2 ): 
            if p.text :
                value = p.text.replace('\xa0', '')
                #prop['Consumer-Rating'] =  value
                C_R = value
                cr = True
            else :
                prop['Consumer-Rating'] =  "N/A"
                C_R = "N/A"
        else :
            break
    if er == False :
      E_R = "N/A"
    if cr == False :
      C_R = False

    # (5) Avalon safety rating
    sr = False
    safety = soup3.find('div', {"class":"css-b7zitt e1pswftg4"})
    if safety :
        S_R = safety.contents[0]
        sr = True
    else :
        S_R = "N/A"
    
    if sr == False :
      S_R = "N/A"
   
    # (6) Avalon Technology
    for  p   in soup3.find('ul', {"class":"css-p619jy-StyledList-List"}).find_all('span', {"class" : "css-lml4x1-ListContent e1jmyeyl2"}) :
        technology.append(p.text)

    #prop['Technology'] = technology

    # (7) Avalon Entertainment: skip Technology
    for  p   in   soup3.find_all('ul', {"class":"css-p619jy-StyledList-List"})[1] :
        for  e   in  p.find_all('span', {"class" : "css-lml4x1-ListContent e1jmyeyl2"}) :
            entertainment.append(e.text)
 
    #prop['Entertainment'] = entertainment
    #L1 = ['Maker', 'Model', 'Year', 'New_Price', 'property']
    #L2 = [maker, model, year, price, prop]
    L1 = ['Maker', 'Model', 'Year', 'New_Price', 'MPG', 'HP', 'Expert-Rating', 'Consumer-Rating', 'Safety-Rating', 'Technology', 'Entertainment']
    L2 = [maker, model, year, price, M_P_G, H_P, E_R, C_R, S_R, technology, entertainment]
    tmp_dct_p = dict(zip(L1, L2))
    print("tmp_dct_p; ", tmp_dct_p)
    return  tmp_dct_p              # return prop dictionary <key : value>

# start of main program
# --------------------------------------------------------------
# select one Maker URL from the Maker list.   For the selected Maker get all
# corresponding models
# read list of dictionaries as the value of maker_dct[Maker].  Each is a URL 
# for Maker page
# read one maker page Url from the makers_list[]
model_dct_list = get_maker_models(makers_List[41])
print("1: list of [model, fyear, fprice, model_url]: " , model_dct_list)
print("\n\n")

# For all models: for each model first year , get the rest of the years data for that model
model_years_dct_list.clear()      # clear model_final_dct[] before populating

# For each model: pass 1st year data and gets back the data for all years for that model
length  = len (model_dct_list)
print("model_dct_list length = ", length)
for    i    in  range (length) :
    tmp_dct.clear()
    tmp_dct = model_dct_list[i].copy()
    maker = tmp_dct.get('Maker')
    model = tmp_dct.get('Model')
    year = tmp_dct.get('Year')
    if i != 0 :
      print ("year = ", year)			# [‘2018’]
      year.replace('[', '')
      year.replace(']', '')
      print ("year = ", year)

    fprice = tmp_dct.get('New_Price')
    model_url = tmp_dct.get('Model-url')
    
    if i == 0 :
        model_years_dct_list.insert(0, get_maker_model_years_price(maker, model, year, fprice, model_url).copy()) 
    else :
        model_years_dct_list.append(get_maker_model_years_price(maker, model, year, fprice, model_url).copy())
            
print("2:  model_years_dct_list[] = : ",  model_years_dct_list)
print("2: length of model_years_dct_list = ", len(model_years_dct_list))
print("done with (2) \n\n")

# for each models/years get that model/year properties into model_final_dct_list[]
# clear model_final_dct[] before populating
model_final_dct_list.clear()                       
length = len (model_years_dct_list) 
print("8: ", length)
sum = 0
ll = len(model_years_dct_list)
for i in range (ll) :
    sum  +=  len(model_years_dct_list[i])
print ("sum = ", sum)
for i  in range (length):
    k = len(model_years_dct_list[i])
    tmp_dct.clear()
    tmp_dct = model_years_dct_list[i].copy()
    for j in range (k) :
        maker = tmp_dct[j].get('Maker')
        model = tmp_dct[j].get('Model')
        year = tmp_dct[j].get('Year')                # List format ['2017']
        #print("year type: ", type(year))
        #print("year: ", year)
        s_year = str(year[0])
        s_year.replace('[', '')
        s_year.replace(']', '')
        #print("s_year: ", s_year)
        price = tmp_dct[j].get('New_Price')
        # get the model URL for the most recent year
        # convert to binary -> subtract one -> convert back to string
        model_year_url = str(tmp_dct[j].get('Model-url'))
        #print("model_year_url: ", model_year_url)
        
        #if  i == 0 and  j == 0 :
        if  j == 0 :
            #print("j = 0 : ", maker, model, year, price, model_year_url)
            #model_final_dct_list.insert(0, copy.deepcopy(get_maker_model_year_price_properties(maker, model, year, price, model_year_url)))
            #model_final_dct_list.insert(0, get_maker_model_year_price_properties(maker, model, year, price, model_year_url).copy())
            model_final_dct_list.append(get_maker_model_year_price_properties(maker, model, year, price, model_year_url))
            print("j = 0: model_final_dct_list length = ", len(model_final_dct_list))
            print("j = 0: ", model_final_dct_list[0], "\n")
        else :
            #print("j != 0 ", maker, model, s_year, price, model_year_url)
            #model_final_dct_list.append(copy.deepcopy(get_maker_model_year_price_properties(maker, model, s_year, price, model_year_url)))
            model_final_dct_list.append(get_maker_model_year_price_properties(maker, model, s_year, price, model_year_url))
            #model_final_dct_list.append(dict(get_maker_model_year_price_properties(maker, model, s_year, price, model_year_url)))
            #print("j != 0: model_final_dct_list length = ", len(model_final_dct_list))
            #print("j != 0: index = 0 ", model_final_dct_list[0], "\n")
            #print("j != 0: index = last entry ", model_final_dct_list[-1], "\n")

print ("3: size of model_final_dct_list = ", len(model_final_dct_list))
print("3: model_final_dct_list[0]", model_final_dct_list[0])
print("3: model_final_dct_list[5]", model_final_dct_list[5])
#print("3: model_final_dct_list[] = : ", model_final_dct_list)
print("\n\n")
# build the data frame for a single Maker
Maker42_df = pd.DataFrame (model_final_dct_list)
print("4: Done\n\n")

# Save Maker1_df.csv on the computer under Adam Home page\Springboard\Makers-DFs folder
#Maker1_df.to_csv (r'C:\Users\Default\Desktop\Maker1_DataFrame.csv', index=False)
Maker42_df.to_csv (r'F:\Users\AdamEzzat\Springboard\Makers-DFs\Maker42_df.csv')

# Read the csv file back as Data Frame in your runtime Cell
# Maker1_df = pd.read_csdone v("C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker1_df.csv")
print("5: CSV is done\n\n")


Maker List size =  44



ymm:  
				2020 Toyota Avalon
			
9:  2020 <class 'str'>
price =  $36,830
ymm:  
				2020 Toyota Avalon Hybrid
			
9:  2020 <class 'str'>
price =  $37,955
ymm:  
				2020 Toyota Camry
			
9:  2020 <class 'str'>
price =  $25,380
ymm:  
				2020 Toyota Camry Hybrid
			
9:  2020 <class 'str'>
price =  $29,385
ymm:  
				2020 Toyota Corolla
			
9:  2020 <class 'str'>
price =  $20,555
ymm:  
				2020 Toyota Corolla Hybrid
			
9:  2020 <class 'str'>
price =  $24,055
ymm:  
				2019 Toyota Mirai
			
9:  2019 <class 'str'>
price =  $59,455
ymm:  
				2020 Toyota Prius
			
9:  2020 <class 'str'>
price =  $25,280
ymm:  
				2020 Toyota Prius Prime
			
9:  2020 <class 'str'>
price =  $28,855
ymm:  
				2020 Toyota Yaris
			
9:  2020 <class 'str'>
price =  $16,605
ymm:  
				2020 Toyota 4Runner
			
9:  2020 <class 'str'>
price =  $37,240
ymm:  
				2020 Toyota C-HR
			
9:  2020 <class 'str'>
price =  $22,415
ymm:  
				2020 Toyota Highlander
			
9:  2020 <class 'str'>
pric

In [0]:
import pandas as pd
Global_df = pd.read_csv('/content/Global_df .csv')

In [0]:
import pandas as pd
Pakistan_df = pd.read_csv('/content/Pakistan_df.csv',  encoding='cp1252')

In [0]:
Global_df['Year'] = Global_df.Year.astype (float)
Global_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 963 entries, 0 to 962
Data columns (total 36 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Unnamed: 0                       963 non-null    int64  
 1   Unnamed: 0.1                     963 non-null    int64  
 2   Unnamed: 0.1.1                   963 non-null    int64  
 3   Maker                            963 non-null    object 
 4   Model                            963 non-null    object 
 5   Year                             963 non-null    float64
 6   New_Price                        963 non-null    object 
 7   MPG                              818 non-null    object 
 8   HP                               917 non-null    object 
 9   Expert-Rating                    963 non-null    float64
 10  Consumer-Rating                  963 non-null    object 
 11  Safety-Rating                    339 non-null    float64
 12  Technology            

In [0]:
KBB_df = Pakistan_df.merge (Global_df, left_on = ['brand', 'model','year'], right_on = ['Maker', 'Model', 'Year'], how = 'inner') 

In [0]:
len(KBB_df)

320

In [0]:
from     bs4 import  BeautifulSoup as soup
import   requests
import   pandas as pd
from     urllib.request  import  urlopen
# Bring CSV files (e.g., Maker1_df.csv) into memory as Data Frame objects (e.g., Maker1_df):
#Maker1_df = pd.read_csv(r'/content/F:\Users\AdamEzzat\Springboard\Makers-DFs\Maker1_df.csv')
#Maker17_df = pd.read_csv(r'/content/F:\Users\AdamEzzat\Springboard\Makers-DFs\Maker17_df.csv')
#Maker18_df = pd.read_csv(r'/content/F:\Users\AdamEzzat\Springboard\Makers-DFs\Maker18_df.csv')
#Maker41_df = pd.read_csv(r'/content/F:\Users\AdamEzzat\Springboard\Makers-DFs\Maker41_df.csv')
#Maker42_df = pd.read_csv(r'/content/F:\Users\AdamEzzat\Springboard\Makers-DFs\Maker42_df.csv')

# Concatenate these Data Frame files into one Data Frame
#df = [ Maker1_df, Maker17_df, Maker18_df, Maker41_df, Maker42_df ] 

# Store this single Data Frame file as CSV file on local disk
#Global_df = pd.concat(df)

# Write CSV files (e.g., Global_df.csv) from the data objects (e.g., Global_df) with expanded 
# Technology & Entertainment:
# We have already the Pakistan_df.csv
#Global_df.to_csv (r'F:\Users\AdamEzzat\Springboard\ BeautifulSoup\Makers-DFs\Global_df.csv')

# Ben’s code to expand the Technology column into multiple columns in the Global_df
# You need to execute each statement below in different cell in Colab
#import ast
#sample_dict = ast.literal_eval(Global_df.head()['property'].iloc[0])
#print(sample_dict)

#Global_df['property_dict']= Global_df['property'].apply(lambda x: ast.literal_eval(x))
#for key in sample_dict: 
    #Global_df[key] = Global_df['property_dict'].apply(lambda x: x[key] if key in x else "ERROR")
#all_tech = set(Global_df['Technology'].sum())
#for tech in all_tech: 
    #Global_df[tech] = Global_df['Technology'].apply(lambda x: 1 if tech in x else 0)
#Global_df

import ast
sample_dict = ast.literal_eval(Global_df.head()['Technology'].iloc[0])
print(sample_dict)

Global_df['Technology_dict']= Global_df['Technology'].apply(lambda x: ast.literal_eval(x))
for key in sample_dict: 
    Global_df[key] = Global_df['Technology_dict'].apply(lambda x: x[key] if key in x else "ERROR")
all_tech = set(Global_df['Technology'].sum())
for tech in all_tech: 
    Global_df[tech] = Global_df['Technology'].apply(lambda x: 1 if tech in x else 0)
Global_df



# Adam’s code to expand the Entertainment column into multiple columns in the Global_df
#all_ent = set(Global_df['Entertainment'].sum())
#for ent in all_ent: 
    #  Global_df[ent] = Global_df['Entertainment'].apply(lambda x: 1 if ent in x else 0)

import ast
sample_dict = ast.literal_eval(Global_df.head()['Technology'].iloc[0])
print(sample_dict)
Global_df['Technology_dict'] = Global_df['Technology'].apply(lambda x: ast.literal_eval(x))
print("Global_df['Technology_dict']: ", Global_df['Technology_dict'])
for key in sample_dict: 
    Global_df[key] = Global_df['Technology_dict'].apply(lambda x: x[key] if key in x else "ERROR")
all_tech = set(Global_df['Technology'].sum())
for tech in all_tech: 
    Global_df[tech] = Global_df['Technology'].apply(lambda x: 1 if tech in x else 0)
Global_df



['Bluetooth Wireless Technology', 'Cruise Control', 'Hands Free Phone', 'Remote Keyless Entry', 'Audible Text Messaging', 'Proximity Sensing Keyless Entry', 'Push-Button Engine Start', 'Smartphone Interface', 'USB Port', 'Voice Recognition System']


TypeError: ignored

# New Section

NameError: ignored

In [0]:
from     bs4 import  BeautifulSoup as soup
import   requests
import   pandas as pd
from     urllib.request  import  urlopen
import   xml.etree.ElementTree  as ET
import   copy

# Create list for Maker’s home page URL
makers_List = ['https://www.kbb.com/acura/' , 'https://www.kbb.com/alfaromeo/' , 'https://www.kbb.com/astonmartin/' , 'https://www.kbb.com/audi/' , 'https://www.kbb.com/bentley/',  'https://www.kbb.com/bmw/' , 'https://www.kbb.com/buick/' , 'https://www.kbb.com/cadillac/' , 'https://www.kbb.com/chevrolet/' , 'https://www.kbb.com/chrysler/' , 'https://www.kbb.com/dodge/' , 'https://www.kbb.com/ferrari/' , 'https://www.kbb.com/fiat/' , 'https://www.kbb.com/ford/' , 'https://www.kbb.com/freightliner/' , 'https://www.kbb.com/genesis/' , 'https://www.kbb.com/gmc/' , 'https://www.kbb.com/honda/' , 'https://www.kbb.com/hyundai/' , 'https://www.kbb.com/infiniti/' , 'https://www.kbb.com/jaguar/' , 'https://www.kbb.com/jeep/' , 'https://www.kbb.com/kia/' , 'https://www.kbb.com/lamborghini/' , 'https://www.kbb.com/landrover/',  'https://www.kbb.com/lexus/' , 'https://www.kbb.com/lincoln/' , 'https://www.kbb.com/maserati/' , 'https://www.kbb.com/mazda/' , 'https://www.kbb.com/mclaren/' , 'https://www.kbb.com/mercedesbenz/' , 'https://www.kbb.com/mini/' , 'https://www.kbb.com/mitsubishi/' , 'https://www.kbb.com/nissan/' , 'https://www.kbb.com/porsche/', 'https://www.kbb.com/ram/' , 'https://www.kbb.com/rivian/' , 'https://www.kbb.com/rollsroyce/' , 'https://www.kbb.com/smart/' , 'https://www.kbb.com/subaru/' , 'https://www.kbb.com/tesla/' , 'https://www.kbb.com/toyota/' , 'https://www.kbb.com/volkswagen/' , 'https://www.kbb.com/volvo/']

# Create list for annual inflation rate in the USA  (2020 - 1964)
# URL: https://www.usinflationcalculator.com/inflation/historical-inflation-rates/  
annual_inflation_rate_List = [0 , 0.018, 0.024, 0.021, 0.013, 0.001, 0.016, 0.015, 0.021, 0.032, 0.016, -0.004, 0.038, 0.028, 0.032, 0.034, 0.027, 0.023, 0.016, 0.028, 0.034, 0.022, 0.016, 0.023, 0.03, 0.028, 0.026, 0.03, 0.042, 0.054, 0.048, 0.041, 0.036, 0.019, 0.036, 0.043, 0.032, 0.062, 0.103, 0.135, 0.113, 0.076, 0.065, 0.058, 0.091, 0.11, 0.062, 0.032, 0.044, 0.057, 0.055, 0.042, 0.031, 0.029, 0.016, 0.013, 0.013]

mListSize = len(makers_List)
print ('Maker List size = ' , mListSize)
print("\n\n")
prop = {}
tmp = {}
tmp_dct_p = {}
tmp_dct = {}
links = []
Maker = ''
first_year = ''
i = 0
m = []
values = []
model_dct_list = []                 # list of all models for a given maker
tmp_dct_list = []       # list of dictionaries structure used by called functions.
model_years_dct_list = []  # holds detailed model info (model, first-year, price, model_url>
model_final_dct_list = []      # final including all years for given model

# input:  maker_url
# output: {Maker : maker, Model : v1,  First Year : v2,  New_Price : v3,  url : link}, {}….] }
def  get_maker_models(maker_url) : 
    r3 = requests.get(maker_url)
    print("maker_url: ", maker_url)
    soup1 = soup( r3.content, 'html.parser')

    # find iterative top level TAG for each maker/model
    link_as = soup1.find('div', {"class" : "make-list-wrapper"}).find_all('div', {"class" : "results-row js-results-row"})
    
    i = 0
    # y is the top-level TAG for 1st year models
    # from that top level get: links, maker, model, first year for all 
    # top-year Models for that Maker’s new Cars
    for   y  in   link_as :       
        x = y.find('a', href=True)
        link = "https://www.kbb.com" + x['href']   # x['href'] has '/Maker/Model'
        if link not in links:
            links.append(link)
            maker, model = (x['href'].strip('/')).split('/')
            Maker = maker[:]
            # get the first year from "2019 Aston Marton DB11"
            ymm = x.find('h3', {"class" : "title-three"}).string 
            print("ymm: ", ymm)            
            m = ymm.split()
            print("9: ", m[0], type(m[0]))                                            # m[0] is 1st year = 2019 is a string
            # first-year = m[0]
            # get the price for the first year
            price = y.find_next('span', {"class" : "title-four"}).text
            print ("price = ", price)
            L1 = ['Maker', 'Model', 'Year', 'New_Price', 'Model-url']
            L2 = [Maker, model, m[0], price, link]
            dct = dict(zip(L1, L2))
            if  i == 0 :
                # copy dct to model_dct_list[0]
                model_dct_list.insert(0, dct.copy())
            else : 
                model_dct_list.append(dct.copy())
            i = i + 1

    print ("\n\n")   
    #print("1: ", model_dct_list) 
    return    model_dct_list                

# Input: model, fyear, fprice, model_url   1st year data for specific maker/model
# Output: [ {model1, year1, price1, model_url1}, {model1, year2, price2, model_url2}, ….]

def   get_maker_model_years_price (maker, model, f_year, f_price, model_url) :
    cur_price = 0
    next_year_price = 0
    r = requests.get(model_url)
    # create soup2 object for the model page for a specific Maker (to search 
    # recursively for <year and Link> )
    soup2 = soup( r.content, 'html.parser')
    # find years and corresponding links for this specific Maker/Model/First-
    # Year/New_Price/Model_url of new Cars
    i = 0
    # convert price (f_price) from string ($xx,xxx) into integer binary representation
    pp = f_price.replace('$', '')
    pp = pp.replace(',', '')
    cur_price = int(pp)                  # convert string pp into integer
    next_year_price = cur_price          # integer binary representation
    
    # write/append first year data for a given model into tmp_dct_list[]
    tmp_dct = {}
    tmp_dct_list.clear()
    L1 = ['Maker', 'Model', 'Year', 'New_Price', 'Model-url']
    L2 = [maker, model, f_year, f_price, model_url]
    tmp_dct = dict(zip(L1, L2))
    # add first year for this specific maker/model
    tmp_dct_list.insert(0, tmp_dct.copy())  
    
    # find index in the annual_inflation_rate_list[] for this model
    if  '2020' in f_year :                  # 2020
        i = 1                               # inflation starts from 2019
    elif  '2019'  in  f_year :              # 2019
        i = 2                               # inflation starts from 2018
    elif   '2018'  in f_year :              # 2018
        i = 3                               # inflation starts from 2017
    else :
        i = -1                              # undefined
        
    last_year = f_year[ : ]
    n = 0                                   # skip first year as I have it already
    # remaining years below
    for  y  in  soup2.find_all("li", {"data-analytics" : True}) : 
        if  n == 0 :             # skip first year as I have it already
            n = n + 1
            continue
        mm_year = y.find_next('a', href=True)# mm_year = “/acura/ilx/2019”
        link = "https://www.kbb.com" + mm_year['href']
        year = y.find_next('a', href=True).contents    # year = “2019” 
        print("type(year): ", type(year))
        print("year: ", year)
        yr = str(year[0])                               # string
        print ("f_year, p_year", f_year, yr)
        if yr == last_year or yr == "" :
            i = i + 1
            continue
        last_year = yr[ : ]
        # estimated cur_price (binary) = 
        #              int(next_year_price  / (1 + annual_inflation_rate_List[i]) )
        # cur_price is integer binary representation
        cur_price = int(next_year_price / (1 + annual_inflation_rate_List[i]) ) 
        next_year_price = cur_price
        p1 = str(cur_price)                # convert binary into string
        l = len(p1) - 3                    # l = (string length – 3)
        p2 = p1[:l] + ',' +  p1[l:]
        p = "$" + p2         # p is current price in string with right format
        #p = "N/A"
        if (len(tmp_dct)) :
            tmp_dct.clear()
        # Needs to adjust the URL by appending year
        m_url = model_url + yr
        L1 = ['Maker', 'Model', 'Year', 'New_Price', 'Model-url']
        L2 = [maker, model, year, p, m_url]
        tmp_dct = dict(zip(L1, L2))         
        tmp_dct_list.append (tmp_dct.copy() )
        i = i + 1            # index into annual inflation rate array
    return  tmp_dct_list     # [ {<maker: <model, year, price, model_url} ]

# input: maker/model/<year : URL> and output: maker/model/years/price/{properties dictionary}
def get_maker_model_year_price_properties(maker, model, year, price, model_year_url) :
    prop.clear()                           # clear the dictionary
    technology = []
    entertainment = []
    #tmp_dct = {}
    i = 0    
    r = requests.get(model_year_url)
    # create soup3 object for the top level HTML page (to search recursively
    # for makers, models and Links)
    soup3 = soup( r.content, 'html.parser')  
    # print(soup3.find('div', {"class" : "css-1wbw7cx epomrkc1"}).find('div', {"class":"stars"}).text)
    # (1-2) find out MPG and HorsePower
    mpg = False
    hp = False
    print("get_maker_model_year_price_properties - year: ", year)

    for p  in soup3.find('div', {"class" : "css-1wbw7cx epomrkc1"}).find_all('div', {"class" : "stars"}):
        parent = str(p.find_parent('div').text).strip()
        #print("parent: ", parent)
        if  "Combined Fuel Economy" in   parent :
            #prop['MPG'] = p.text
            M_P_G = p.text
            mpg = True
        elif  "Horsepower"  in   parent :
            #prop['HP'] = p.text
            H_P = p.text
            hp = True
        else :
            break
    if mpg == False :
        #prop['MPG'] = "N/A"
        M_P_G = "N/A"
    if hp == False :
          #prop['HP'] = "N/A"
          H_P = "N/A"

    # (3-4) find out Expert Rating and Consumer rating
    i = 0
    er = False
    cr = False
    for p in soup3.find_all('div', {"class":"css-x9skgx-NumericRating e149jidm1"}) :
        i = i + 1
        if i == 1 :
            if  p.text :
                value = p.text.replace('\xa0', '')
                #prop['Expert-Rating'] = value
                E_R = value
                er = True
            else :
                prop['Expert-Rating'] = "N/A"
                E_R = "N/A"
        elif (i == 2 ): 
            if p.text :
                value = p.text.replace('\xa0', '')
                #prop['Consumer-Rating'] =  value
                C_R = value
                cr = True
            else :
                prop['Consumer-Rating'] =  "N/A"
                C_R = "N/A"
        else :
            break
    if er == False :
      E_R = "N/A"
    if cr == False :
      C_R = False

    # (5) Avalon safety rating
    sr = False
    safety = soup3.find('div', {"class":"css-b7zitt e1pswftg4"})
    if safety :
        S_R = safety.contents[0]
        sr = True
    else :
        S_R = "N/A"
    
    if sr == False :
      S_R = "N/A"
   
    # (6) Avalon Technology
    for  p   in soup3.find('ul', {"class":"css-p619jy-StyledList-List"}).find_all('span', {"class" : "css-lml4x1-ListContent e1jmyeyl2"}) :
        technology.append(p.text)

    #prop['Technology'] = technology

    # (7) Avalon Entertainment: skip Technology
    for  p   in   soup3.find_all('ul', {"class":"css-p619jy-StyledList-List"})[1] :
        for  e   in  p.find_all('span', {"class" : "css-lml4x1-ListContent e1jmyeyl2"}) :
            entertainment.append(e.text)
 
    #prop['Entertainment'] = entertainment
    #L1 = ['Maker', 'Model', 'Year', 'New_Price', 'property']
    #L2 = [maker, model, year, price, prop]
    L1 = ['Maker', 'Model', 'Year', 'New_Price', 'MPG', 'HP', 'Expert-Rating', 'Consumer-Rating', 'Safety-Rating', 'Technology', 'Entertainment']
    L2 = [maker, model, year, price, M_P_G, H_P, E_R, C_R, S_R, technology, entertainment]
    tmp_dct_p = dict(zip(L1, L2))
    print("tmp_dct_p; ", tmp_dct_p)
    return  tmp_dct_p              # return prop dictionary <key : value>

# start of main program
# --------------------------------------------------------------
# select one Maker URL from the Maker list.   For the selected Maker get all
# corresponding models
# read list of dictionaries as the value of maker_dct[Maker].  Each is a URL 
# for Maker page
# read one maker page Url from the makers_list[]
model_dct_list = get_maker_models(makers_List[41])
print("1: list of [model, fyear, fprice, model_url]: " , model_dct_list)
print("\n\n")

# For all models: for each model first year , get the rest of the years data for that model
model_years_dct_list.clear()      # clear model_final_dct[] before populating

# For each model: pass 1st year data and gets back the data for all years for that model
length  = len (model_dct_list)
print("model_dct_list length = ", length)
for    i    in  range (length) :
    tmp_dct.clear()
    tmp_dct = model_dct_list[i].copy()
    maker = tmp_dct.get('Maker')
    model = tmp_dct.get('Model')
    year = tmp_dct.get('Year')
    if i != 0 :
      print ("year = ", year)     # [‘2018’]
      year.replace('[', '')
      year.replace(']', '')
      print ("year = ", year)

    fprice = tmp_dct.get('New_Price')
    model_url = tmp_dct.get('Model-url')
    
    if i == 0 :
        model_years_dct_list.insert(0, get_maker_model_years_price(maker, model, year, fprice, model_url).copy()) 
    else :
        model_years_dct_list.append(get_maker_model_years_price(maker, model, year, fprice, model_url).copy())
            
print("2:  model_years_dct_list[] = : ",  model_years_dct_list)
print("2: length of model_years_dct_list = ", len(model_years_dct_list))
print("done with (2) \n\n")

# for each models/years get that model/year properties into model_final_dct_list[]
# clear model_final_dct[] before populating
model_final_dct_list.clear()                       
length = len (model_years_dct_list) 
print("8: ", length)
sum = 0
ll = len(model_years_dct_list)
for i in range (ll) :
    sum  +=  len(model_years_dct_list[i])
print ("sum = ", sum)
for i  in range (length):
    k = len(model_years_dct_list[i])
    tmp_dct.clear()
    tmp_dct = model_years_dct_list[i].copy()
    for j in range (k) :
        maker = tmp_dct[j].get('Maker')
        model = tmp_dct[j].get('Model')
        year = tmp_dct[j].get('Year')                # List format ['2017']
        print("year type: ", type(year))
        print("year: ", year)
        s_year = str(year[0])
        s_year.replace('[', '')
        s_year.replace(']', '')
        print("s_year: ", s_year)
        price = tmp_dct[j].get('New_Price')
        # get the model URL for the most recent year
        # convert to binary -> subtract one -> convert back to string
        model_year_url = str(tmp_dct[j].get('Model-url'))
        print("model_year_url: ", model_year_url)
        
        #if  i == 0 and  j == 0 :
        if  j == 0 :
            print("j = 0 : ", maker, model, year, price, model_year_url)
            #model_final_dct_list.insert(0, get_maker_model_year_price_properties(maker, model, year, price, model_year_url).copy())
            model_final_dct_list.append(get_maker_model_year_price_properties(maker, model, year, price, model_year_url))
            print("j = 0: model_final_dct_list length = ", len(model_final_dct_list))
            print("j = 0: ", model_final_dct_list[0], "\n")
        else :
            print("j != 0 ", maker, model, s_year, price, model_year_url)
            #model_final_dct_list.append(copy.deepcopy(get_maker_model_year_price_properties(maker, model, s_year, price, model_year_url)))
            model_final_dct_list.append(get_maker_model_year_price_properties(maker, model, s_year, price, model_year_url))
            print("j != 0: model_final_dct_list length = ", len(model_final_dct_list))
            print("j != 0: index = 0 ", model_final_dct_list[0], "\n")
            print("j != 0: index = last entry ", model_final_dct_list[-1], "\n")

print ("3: size of model_final_dct_list = ", len(model_final_dct_list))
print("3: model_final_dct_list[0]", model_final_dct_list[0])
print("3: model_final_dct_list[5]", model_final_dct_list[5])
print("3: model_final_dct_list[] = : ", model_final_dct_list)
print("\n\n")
# build the data frame for a single Maker
Maker42_df = pd.DataFrame (model_final_dct_list)
print("4: Done\n\n")

# Save Maker1_df.csv on the computer under Adam Home page\Springboard\Makers-DFs folder
#Maker1_df.to_csv (r'C:\Users\Default\Desktop\Maker1_DataFrame.csv', index=False)
Maker42_df.to_csv (r'F:\Users\AdamEzzat\Springboard\Makers-DFs\Maker42_df.csv')

# Read the csv file back as Data Frame in your runtime Cell
# Maker1_df = pd.read_csdone v("C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker1_df.csv")
print("5: CSV is done\n\n")


Streaming output truncated to the last 5000 lines.
f_year, p_year 2020 2005
type(year):  <class 'list'>
year:  ['2004']
f_year, p_year 2020 2004
type(year):  <class 'list'>
year:  ['2003']
f_year, p_year 2020 2003
type(year):  <class 'list'>
year:  ['2002']
f_year, p_year 2020 2002
type(year):  <class 'list'>
year:  ['2001']
f_year, p_year 2020 2001
year =  2020
year =  2020
type(year):  <class 'list'>
year:  ['2019']
f_year, p_year 2020 2019
type(year):  <class 'list'>
year:  ['2018']
f_year, p_year 2020 2018
type(year):  <class 'list'>
year:  ['2017']
f_year, p_year 2020 2017
year =  2020
year =  2020
type(year):  <class 'list'>
year:  ['2019']
f_year, p_year 2020 2019
type(year):  <class 'list'>
year:  ['2018']
f_year, p_year 2020 2018
type(year):  <class 'list'>
year:  ['2017']
f_year, p_year 2020 2017
type(year):  <class 'list'>
year:  ['2016']
f_year, p_year 2020 2016
type(year):  <class 'list'>
year:  ['2015']
f_year, p_year 2020 2015
type(year):  <class 'list'>
year:  ['2014']


In [35]:
# This is current copy to expand both the Technology and Entertainment columns
# in the Global_df data frame and save it as CSV file on local disk (Global_df.csv

from     bs4 import  BeautifulSoup as soup
import   requests
import   pandas as pd
from     urllib.request  import  urlopen
import   copy
import   ast

sample_dict = []

Global_df = pd.read_csv(r'/content/Global_df.csv')
Global_df

# Adam’s code to expand the Technology column into multiple columns in the Global_df
# # longest Technology list is row 1595 ==> 16 columns with one redundant column (last)
#sample_dict = ast.literal_eval(Global_df.head()['Technology'].iloc[0])   # this is a list
print("Technoloy type: ", type(Global_df['Technology']))
sample_dict = ast.literal_eval(Global_df['Technology'].iloc[1595]) 
sample_dict.pop( )                                    # remove last element
sample_dict.append('Voice Recognition System')
print("0: Sample dict[0] ", sample_dict[0])          # list of all Technology columns/keys
print("0: sample_dict", sample_dict)
print("0: sample_dict type ", type(sample_dict))
print("0: sample_dict[0] type: ", type(sample_dict[0]))
#print("1: len of sample_dict", len(sample_dict))      # length = 15 elements

print(Global_df)
print(len(Global_df.index))

for i  in  range(len(Global_df.index)) :    # row in Global_data frame
  ll = Global_df['Technology'].iloc[i]      # evaluate Technoloy list for row i
  ll = ast.literal_eval(ll)                 # cast it from string to list of strings
  print("2: i = row # ", i, "ll: Tech length: ", len(ll), "ll Tech list: ", ll)         # list of technology columns

#get all the list from technology from tech column and then combine in into one big list and then take a set of that list to the unique values  and loop through each value then GLobabl_df[column name ] = 0 
  for j  in  range(len(sample_dict)) :           # each column (key) in sample_dict
    #print("2: sample_dict[j] = ", sample_dict[j])
    #print("2: sample_dict type ", type(sample_dict))
    #print("2: sample_dict[0] type: ", type(sample_dict[0]))
    #print("sample_dict length = ", len(sample_dict))
    print("2: row i = ", i, "Tech length = ", len(ll))
    print("2: j = ", j, "column len = ", len(ll[j]))   # print column name - key
    if sample_dict[j] in  ll :
      print("found1 column in Tech: ", sample_dict[j], "\n")
      Global_df[sample_dict[j]].iloc[i] = 1
      print("found2 column in Tech\n")
      print("3: column = ", sample_dict[j], "value = ", 1)
    else :
      print("Column is missing from Tech\n")
      Global_df[sample_dict[j]].iloc[i]= 0

print("Global_df expanding Technology: ", Global_df)

###############################################################
# Adam new version of the code to expand Entertainment column
# longest Entertainment list is row 4 ==> 7 columns ????

sample_dict1 = ['iPod Connector', 'MP3 Player', 'Premium Radio', 'Satellite Radio', 'Bluetooth Streaming Audio', 'CD Player', 'iPod Interface']

for i  in  range(len(Global_df.index))  :
  ll = Global_df['Entertainment'].iloc[i]
  ll = ast.literal_eval(ll)      # cast it from string to list
  print("4: i + row# = ", i, "ll Entertainment length = ", len(ll), "Entertainment: ", ll)


for i  in  range(len(Global_df.index)) :       # row# in the Global data frame
  print(i)
  ll = Global_df['Entertainment'].iloc[i]     # Entertainment list for row i
  ll = ast.literal_eval(ll)
  print("5: i = row # = ", i, " and list length = ", len(ll), "list: ", ll)
  
  for j  in  range(len(sample_dict1)) :           # each column (key) in sample_dict (1)
    #print("5: sample_dict[j] = ", sample_dict1[j])
    if sample_dict1[j] in  ll :
      #print("3: found element: ", sample_dict[j])
      Global_df[sample_dict1[j]].iloc[i] = 1
      #print("4: found element: ", sample_dict[j])
    else :
      #print("3: did not find element: ", sample_dict1[j])
      Global_df[sample_dict1[j]].iloc[i] = 0
      print("4: column = ", sample_dict1[j], "value = ", 0)

#print("Global_df expanding Entertainment: ", Global_df)
#for i  in range(10, 50, 5) :
  #print("15: i = ", i, "Global_df[i]: ", Global_df.iloc[i])

# print both Technology colmns + Entertainment columns for row 4
print("5: Bluetooth Wireless Technology: ", Global_df['Bluetooth Wireless Technology'].iloc[4])
#print("5: Remote Engine Start: ", Global_df['Remote Engine Start'].iloc[4])
#print("5: row = 4, Technology: ", Global_df['Technology'].iloc[4])
#print("5: row = 4, Technology: ", Global_df['Bluetooth Wireless Technology'].iloc[4], Global_df['Cruise Control'].iloc[4], Global_df['Hands Free Phone'].iloc[4], Global_df['Navigation System'].iloc[4], Global_df['Remote Keyless Entry'].iloc[4], Global_df['Audible Text Messaging'].iloc[4], Global_df['Proximity Sensing Keyless Entry'].iloc[4], Global_df['Push-Button Engine Start'].iloc[4], Global_df['Real-Time Traffic Information'].iloc[4], Global_df['Remote Engine Start'].iloc[4], Global_df['Smartphone Interface'].iloc[4], Global_df['USB Port'].iloc[4], Global_df['Voice Recognition System'].iloc[4] )
#print("5: row = 4, Entertainment: ", Global_df['Entertainment'].iloc[4])
print("5: row = 4, Entertainment: ", Global_df['iPod Connector'].iloc[4], Global_df['MP3 Player'].iloc[4], Global_df['Premium Radio'].iloc[4], Global_df['Satellite Radio'].iloc[4], Global_df['Bluetooth Streaming Audio'].iloc[4], Global_df['CD Player'].iloc[4], Global_df['iPod Interface'].iloc[4] )

#print("6: row = 960, Technology: ", Global_df['Technology'].iloc[960])
#print("6: row = 960, Entertainment: ", Global_df['Entertainment'].iloc[960])

i = 707
print("7: row = i = ", i, "Technology: ", Global_df['Bluetooth Wireless Technology'].iloc[i], Global_df['Cruise Control'].iloc[i], Global_df['Hands Free Phone'].iloc[i], Global_df['Navigation System'].iloc[i], Global_df['Remote Keyless Entry'].iloc[i], Global_df['Audible Text Messaging'].iloc[i], Global_df['Internet Access'].iloc[i], Global_df['Proximity Sensing Keyless Entry'].iloc[i], Global_df['Push-Button Engine Start'].iloc[i], Global_df['Real-Time Traffic Information'].iloc[i], Global_df['Remote Engine Start'].iloc[i], Global_df['Smartphone Interface'].iloc[i], Global_df['USB Port'].iloc[i], Global_df['Voice Recognition System'].iloc[i], Global_df['Touch Screen Monitor'].iloc[i] )
print("7: row = i = ", i, "Entertainment: ", Global_df['iPod Connector'].iloc[i], Global_df['MP3 Player'].iloc[i], Global_df['Premium Radio'].iloc[i], Global_df['Satellite Radio'].iloc[i], Global_df['Bluetooth Streaming Audio'].iloc[i], Global_df['CD Player'].iloc[i], Global_df['iPod Interface'].iloc[i])


Technoloy type:  <class 'pandas.core.series.Series'>
0: Sample dict[0]  Bluetooth Wireless Technology
0: sample_dict ['Bluetooth Wireless Technology', 'Cruise Control', 'Hands Free Phone', 'Navigation System', 'Remote Keyless Entry', 'Audible Text Messaging', 'Internet Access', 'Proximity Sensing Keyless Entry', 'Push-Button Engine Start', 'Real-Time Traffic Information', 'Remote Engine Start', 'Smartphone Interface', 'Touch Screen Monitor', 'USB Port', 'Voice Recognition System']
0: sample_dict type  <class 'list'>
0: sample_dict[0] type:  <class 'str'>
      Unnamed: 0  ...                                      Entertainment
0              0  ...  ['iPod Connector', 'MP3 Player', 'Satellite Ra...
1              1  ...  ['iPod Connector', 'MP3 Player', 'Bluetooth St...
2              2  ...  ['iPod Connector', 'MP3 Player', 'Satellite Ra...
3              3  ...  ['iPod Connector', 'MP3 Player', 'Premium Radi...
4              4  ...  ['iPod Connector', 'MP3 Player', 'Premium Radi...
.

KeyError: ignored

In [0]:
Global_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2612 entries, 0 to 2611
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       2612 non-null   int64  
 1   Unnamed: 0.1     2214 non-null   float64
 2   Maker            2612 non-null   object 
 3   Model            2612 non-null   object 
 4   Year             2612 non-null   int64  
 5   New_Price        2612 non-null   object 
 6   MPG              2401 non-null   object 
 7   HP               2533 non-null   object 
 8   Expert-Rating    2609 non-null   float64
 9   Consumer-Rating  2612 non-null   object 
 10  Safety-Rating    755 non-null    float64
 11  Technology       2612 non-null   object 
 12  Entertainment    2612 non-null   object 
dtypes: float64(3), int64(2), object(8)
memory usage: 265.4+ KB


In [0]:
# This is current copy
from     bs4 import  BeautifulSoup as soup
import   requests
import   pandas as pd
from     urllib.request  import  urlopen
import   copy
import   ast


Global_df = pd.read_csv(r'/content/Global_df.csv')
print("0: Global_df: ", Global_df)
print("0: type(Global_df): ", type(Global_df))
print("Global_df row 1595 Tech column: ", Global_df['Technology'].iloc[1595])


# Adam’s code to expand the Technology column into multiple columns in the Global_df
# longest Technology list is row 707 ==> 12 columns
# We need to add 3 columns => union of all column in Tech = 15 columns

import ast
#sample_dict = ast.literal_eval(Global_df.head()['Technology'].iloc[0])   # this is a list
sample_dict = ast.literal_eval(Global_df['Technology'].iloc[707]) 
sample_dict.append('Remote Engine Start')
print("0: Sample Dict ", sample_dict)                  # list of all Technology columns/keys
#print("1: len of sample_dict", len(sample_dict))      # length = 15 elements
#print("1: ", sample_dict, "\n\n")
#print("1: length of Global_df.index: ", len(Global_df.index))
#print("1: row = 5, Technology: ", Global_df['Technology'].iloc[5])

for i  in  range(len(Global_df.index)) :    # row in Global_data frame
  ll = Global_df['Technology'].iloc[i]      # evaluate Technoloy list for row i
  ll = ast.literal_eval(ll)                 # cast it from string to list of strings
  print("2: i = row # = ", i, " and ll = ", ll)
  #print("2: i = row # ", i, "ll: Tech length: ", len(ll), "ll Tech list: ", ll)         # list of technology columns
  # Copy ll[] into lst[]
  #lst = copy.copy(ll) 
  #lst = ll.copy()
  #lst = ll[ : ]                          # copy all elements from ll[] to lst[]
  #print ("type(lst) = ", type(lst))
  #print("2: i = row # = ", i, " and lst = ", lst)
  #print("2: i = row # = ", i, " and list length = ", len(lst), "list: ", lst)
  
  for j  in  range(len(sample_dict)) :           # each column (key) in sample_dict
    print("2: sample_dict[j] = ", sample_dict[j])   # print column name - key
    if sample_dict[j] in  ll :
      Global_df[sample_dict[j]].iloc[i] = 1
      #print("3: column = ", sample_dict[j], "value = ", 1)
    else :
      Global_df[sample_dict[j]].iloc[i]= 0
      #print("3: column = ", sample_dict[j], "value = ", 0)

#print("Global_df expanding Technology: ", Global_df)

# Adam new version of the code to expand Entertainment column
# longest Entertainment list is row 4 ==> 7 columns
#for i  in  range(len(Global_df.index))  :
  #ll = Global_df['Entertainment'].iloc[i]
  #ll = ast.literal_eval(ll)      # cast it from string to list
  #print("4: i + row# = ", i, "ll Entertainment length = ", len(ll), "Entertainment: ", ll)

sample_dict1 = ast.literal_eval(Global_df['Entertainment'].iloc[4])   # this is a list
#print("4: len of sample_dict1", len(sample_dict1))
#print("4: ", sample_dict1, "\n\n")
#print("4: length of Global_df.index: ", len(Global_df.index))
#print("4: row = 5, Entertainment: ", Global_df['Entertainment'].iloc[5])

for i  in  range(len(Global_df.index)) :       # row # in the Global data frame
  ll = Global_df['Entertainment'].iloc[i]     # Entertainment list for row i
  ll = ast.literal_eval(ll)
  #print("5: i = row # = ", i, " and list length = ", len(lst), "list: ", lst)
  for j  in  range(len(sample_dict1)) :           # each column (key) in sample_dict
    #print("5: sample_dict[j] = ", sample_dict1[j])
    if sample_dict1[j] in  ll :
      #print("6: found element: ", sample_dict[j])
      Global_df[sample_dict1[j]].iloc[i] = 1
      #print("6: column = ", sample_dict1[j], "value = ", 1)
    else :
      #print("3: did not find element: ", sample_dict1[j])
      Global_df[sample_dict1[j]].iloc[i] = 0
      #print("6: column = ", sample_dict1[j], "value = ", 0)

#print("Global_df expanding Entertainment: ", Global_df)
#for i  in range(10, 50, 5) :
  #print("15: i = ", i, "Global_df[i]: ", Global_df.iloc[i])

# print both Technology colmns + Entertainment columns for row 4
print("7: Bluetooth Wireless Technology: ", Global_df['Bluetooth Wireless Technology'].iloc[4])
#print("7: Remote Engine Start: ", Global_df['Remote Engine Start'].iloc[4])
#print("7: row = 4, Technology: ", Global_df['Technology'].iloc[4])
#print("7: row = 4, Technology: ", Global_df['Bluetooth Wireless Technology'].iloc[4], Global_df['Cruise Control'].iloc[4], Global_df['Hands Free Phone'].iloc[4], Global_df['Navigation System'].iloc[4], Global_df['Remote Keyless Entry'].iloc[4], Global_df['Audible Text Messaging'].iloc[4], Global_df['Proximity Sensing Keyless Entry'].iloc[4], Global_df['Push-Button Engine Start'].iloc[4], Global_df['Real-Time Traffic Information'].iloc[4], Global_df['Remote Engine Start'].iloc[4], Global_df['Smartphone Interface'].iloc[4], Global_df['USB Port'].iloc[4], Global_df['Voice Recognition System'].iloc[4] )
#print("7: row = 4, Entertainment: ", Global_df['Entertainment'].iloc[4])
print("7: row = 4, Entertainment: ", Global_df['iPod Connector'].iloc[4], Global_df['MP3 Player'].iloc[4], Global_df['Premium Radio'].iloc[4], Global_df['Satellite Radio'].iloc[4], Global_df['Bluetooth Streaming Audio'].iloc[4], Global_df['CD Player'].iloc[4], Global_df['iPod Interface'].iloc[4] )

#print("7: row = 960, Technology: ", Global_df['Technology'].iloc[960])
#print("7: row = 960, Entertainment: ", Global_df['Entertainment'].iloc[960])

i = 707
print("7: row = i = ", i, "Technology: ", Global_df['Bluetooth Wireless Technology'].iloc[i], Global_df['Cruise Control'].iloc[i], Global_df['Hands Free Phone'].iloc[i], Global_df['Navigation System'].iloc[i], Global_df['Remote Keyless Entry'].iloc[i], Global_df['Audible Text Messaging'].iloc[i], Global_df['Internet Access'].iloc[i], Global_df['Proximity Sensing Keyless Entry'].iloc[i], Global_df['Push-Button Engine Start'].iloc[i], Global_df['Real-Time Traffic Information'].iloc[i], Global_df['Remote Engine Start'].iloc[i], Global_df['Smartphone Interface'].iloc[i], Global_df['USB Port'].iloc[i], Global_df['Voice Recognition System'].iloc[i], Global_df['Touch Screen Monitor'].iloc[i] )
print("7: row = i = ", i, "Entertainment: ", Global_df['iPod Connector'].iloc[i], Global_df['MP3 Player'].iloc[i], Global_df['Premium Radio'].iloc[i], Global_df['Satellite Radio'].iloc[i], Global_df['Bluetooth Streaming Audio'].iloc[i], Global_df['CD Player'].iloc[i], Global_df['iPod Interface'].iloc[i])

0: Global_df:        Unnamed: 0  ...                                      Entertainment
0              0  ...  ['iPod Connector', 'MP3 Player', 'Satellite Ra...
1              1  ...  ['iPod Connector', 'MP3 Player', 'Bluetooth St...
2              2  ...  ['iPod Connector', 'MP3 Player', 'Satellite Ra...
3              3  ...  ['iPod Connector', 'MP3 Player', 'Premium Radi...
4              4  ...  ['iPod Connector', 'MP3 Player', 'Premium Radi...
...          ...  ...                                                ...
2607         368  ...  ['iPod Connector', 'Premium Radio', 'Bluetooth...
2608         369  ...  ['iPod Connector', 'Premium Radio', 'Satellite...
2609         370  ...   ['Satellite Radio', 'Bluetooth Streaming Audio']
2610         371  ...  ['iPod Connector', 'MP3 Player', 'Bluetooth St...
2611         372  ...  ['iPod Connector', 'Satellite Radio', 'Bluetoo...

[2612 rows x 13 columns]
0: type(Global_df):  <class 'pandas.core.frame.DataFrame'>
Global_df row 1595 Tech 

KeyError: ignored

In [0]:
print(Global_df)

      Unnamed: 0  ...                                      Entertainment
0              0  ...  ['iPod Connector', 'MP3 Player', 'Satellite Ra...
1              1  ...  ['iPod Connector', 'MP3 Player', 'Bluetooth St...
2              2  ...  ['iPod Connector', 'MP3 Player', 'Satellite Ra...
3              3  ...  ['iPod Connector', 'MP3 Player', 'Premium Radi...
4              4  ...  ['iPod Connector', 'MP3 Player', 'Premium Radi...
...          ...  ...                                                ...
2607         368  ...  ['iPod Connector', 'Premium Radio', 'Bluetooth...
2608         369  ...  ['iPod Connector', 'Premium Radio', 'Satellite...
2609         370  ...   ['Satellite Radio', 'Bluetooth Streaming Audio']
2610         371  ...  ['iPod Connector', 'MP3 Player', 'Bluetooth St...
2611         372  ...  ['iPod Connector', 'Satellite Radio', 'Bluetoo...

[2612 rows x 13 columns]


In [0]:
# This program is to concatenate set of Makers data frame into Global_df
# and store it as CSV file on local disk (Global_df.csv

from     bs4 import  BeautifulSoup as soup
import   requests
import   pandas as pd
from     urllib.request  import  urlopen
from     google.colab  import files

# Bring CSV files (e.g., Maker1_df.csv) into memory as Data Frame objects (e.g., Maker1_df):
Maker1_df = pd.read_csv(r'/content/F__Users_AdamEzzat_Springboard_Makers-DFs_Maker1_df.csv')
Maker4_df = pd.read_csv(r'/content/F__Users_AdamEzzat_Springboard_Makers-DFs_Maker4_df.csv')
Maker6_df = pd.read_csv(r'/content/F__Users_AdamEzzat_Springboard_Makers-DFs_Maker6_df.csv')
Maker9_df = pd.read_csv(r'/content/F__Users_AdamEzzat_Springboard_Makers-DFs_Maker9_df.csv')
Maker18_df = pd.read_csv(r'/content/F__Users_AdamEzzat_Springboard_Makers-DFs_Maker18_df.csv')
Maker19_df = pd.read_csv(r'/content/F__Users_AdamEzzat_Springboard_Makers-DFs_Maker19_df.csv')
Maker23_df = pd.read_csv(r'/content/F__Users_AdamEzzat_Springboard_Makers-DFs_Maker23_df.csv')
Maker26_df = pd.read_csv(r'/content/F__Users_AdamEzzat_Springboard_Makers-DFs_Maker26_df.csv')
Maker29_df = pd.read_csv(r'/content/F__Users_AdamEzzat_Springboard_Makers-DFs_Maker29_df.csv')
Maker42_df = pd.read_csv(r'/content/F__Users_AdamEzzat_Springboard_Makers-DFs_Maker42_df.csv')

# Concatenate these Data Frame files into one Data Frame
df = [ Maker1_df, Maker4_df, Maker6_df, Maker9_df, Maker18_df, Maker19_df, Maker23_df, Maker26_df, Maker29_df, Maker33_df, Maker34_df, Maker42_df ] 

list = []

# Store this single Data Frame file as CSV file on local disk
Global_df = pd.concat(df)
print("0: Global_df: ", Global_df)
print("0: type(Global_df): ", type(Global_df))

# Write CSV files (e.g., Global_df.csv) from the data objects (e.g., Global_df) with expanded 
# Technology & Entertainment:
# We have already the Pakistan_df.csv
#Global_df.to_csv (r'F:\Users\AdamEzzat\Springboard\BeautifulSoup\Makers-DFs\Global_df.csv')
Global_df.to_csv('Global_df.csv')
files.download('Global_df.csv')      # now on Colab LHS download the CSV file to disk


0: Global_df:       Unnamed: 0  ...                                      Entertainment
0           0.0  ...  ['iPod Connector', 'MP3 Player', 'Satellite Ra...
1           1.0  ...  ['iPod Connector', 'MP3 Player', 'Bluetooth St...
2           2.0  ...  ['iPod Connector', 'MP3 Player', 'Satellite Ra...
3           3.0  ...  ['iPod Connector', 'MP3 Player', 'Premium Radi...
4           4.0  ...  ['iPod Connector', 'MP3 Player', 'Premium Radi...
..          ...  ...                                                ...
368       368.0  ...  ['iPod Connector', 'Premium Radio', 'Bluetooth...
369       369.0  ...  ['iPod Connector', 'Premium Radio', 'Satellite...
370       370.0  ...   ['Satellite Radio', 'Bluetooth Streaming Audio']
371       371.0  ...  ['iPod Connector', 'MP3 Player', 'Bluetooth St...
372       372.0  ...  ['iPod Connector', 'Satellite Radio', 'Bluetoo...

[2612 rows x 12 columns]
0: type(Global_df):  <class 'pandas.core.frame.DataFrame'>


In [0]:
from google.colab import files

Global_df.to_csv('Global_df.csv')
files.download('Global_df.csv')

In [0]:
print(Global_df)


     Unnamed: 0  Unnamed: 0.1  ... iPod Interface Remote Engine Start
0             0             0  ...              0                   0
1             1             1  ...              0                   0
2             2             2  ...              0                   0
3             3             3  ...              0                   0
4             4             4  ...              0                   0
..          ...           ...  ...            ...                 ...
958         368           368  ...              0                   0
959         369           369  ...              0                   0
960         370           370  ...              0                   0
961         371           371  ...              0                   0
962         372           372  ...              0                   0

[963 rows x 35 columns]


In [0]:
Global_df = pd.read_csv("/content/Global_df.csv")


In [0]:
print(Global_df)

     Unnamed: 0  Unnamed: 0.1  ... CD Player iPod Interface
0             0             0  ...         0              0
1             1             1  ...         0              0
2             2             2  ...         0              0
3             3             3  ...         0              0
4             4             4  ...         0              0
..          ...           ...  ...       ...            ...
958         368           368  ...         0              0
959         369           369  ...         0              0
960         370           370  ...         0              0
961         371           371  ...         0              0
962         372           372  ...         0              0

[963 rows x 34 columns]


In [0]:
print (Global_df)

NameError: ignored

In [0]:
from google.colab import files
Global_df.to_csv('Global_df.csv')
files.download('Global_df.csv')

In [0]:
import pandas as pd
from google.colab   import files
Global_df.to_csv('Global_df.csv')
files.download('Global_df.csv')

In [0]:
Pakistan_df = pd.read_csv('Pakistan_df.csv' , encoding='cp1252')

In [0]:
Global_df['Year'] = Global_df.Year.astype (float)
Global_df.info()                                                                

<class 'pandas.core.frame.DataFrame'>
Int64Index: 963 entries, 0 to 372
Data columns (total 28 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Unnamed: 0                       963 non-null    int64  
 1   Maker                            963 non-null    object 
 2   Model                            963 non-null    object 
 3   Year                             963 non-null    float64
 4   New_Price                        963 non-null    object 
 5   MPG                              818 non-null    object 
 6   HP                               917 non-null    object 
 7   Expert-Rating                    963 non-null    float64
 8   Consumer-Rating                  963 non-null    object 
 9   Safety-Rating                    339 non-null    float64
 10  Technology                       963 non-null    object 
 11  Entertainment                    963 non-null    object 
 12  Bluetooth Wireless Tec

In [0]:
KBB_df = Pakistan_df.merge (Global_df, left_on = ['Brand', 'Model','Year'], right_on = ['Maker', 'Model', 'Year'], how = 'inner') 

In [0]:
from google.colab  import files
KBB_df.to_csv('KBB.csv')
files.download('KBB.csv')

In [0]:
Global_df = pd.read_csv('/content/Global_df.csv')

In [0]:
Global_df['Year'] = Global_df.Year.astype (float)
Global_df.info()                                                                

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 963 entries, 0 to 962
Data columns (total 29 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Unnamed: 0                       963 non-null    int64  
 1   Unnamed: 0.1                     963 non-null    int64  
 2   Maker                            963 non-null    object 
 3   Model                            963 non-null    object 
 4   Year                             963 non-null    float64
 5   New_Price                        963 non-null    object 
 6   MPG                              818 non-null    object 
 7   HP                               917 non-null    object 
 8   Expert-Rating                    963 non-null    float64
 9   Consumer-Rating                  963 non-null    object 
 10  Safety-Rating                    339 non-null    float64
 11  Technology                       963 non-null    object 
 12  Entertainment         

In [0]:
KBB_df = Pakistan_df.merge (Global_df, left_on = ['Brand', 'Model','Year'], right_on = ['Maker', 'Model', 'Year'], how = 'inn

In [0]:
print(KBB_df)

      Brand Condition  ... CD Player  iPod Interface
0    Toyota      Used  ...         0               0
1    Toyota      Used  ...         0               0
2    Toyota      Used  ...         0               0
3    Toyota      Used  ...         0               0
4    Toyota       New  ...         0               0
..      ...       ...  ...       ...             ...
312   Honda      Used  ...         0               0
313   Honda      Used  ...         0               0
314  Toyota      Used  ...         0               0
315  Toyota      Used  ...         0               0
316  Toyota      Used  ...         0               0

[317 rows x 36 columns]


In [0]:
from google.colab  import files
KBB_df.to_csv('KBB_df.csv')
files.download('KBB_df.csv')

In [0]:
Global_df

,Unnamed: 0,Maker,Model,Year,New_Price,MPG,HP,Expert-Rating,Consumer-Rating,Safety-Rating,Technology,Entertainment,Bluetooth Wireless Technology,Cruise Control,Hands Free Phone,Remote Keyless Entry,Audible Text Messaging,Proximity Sensing Keyless Entry,Push-Button Engine Start,Smartphone Interface,USB Port,Voice Recognition System,iPod Connector,MP3 Player,Satellite Radio,Bluetooth Streaming Audio,CD Player,iPod Interface
0,0,acura,ilx,2020,"$26,925",28 MPG,201 HP,4.4,4.4,5.0,"['Bluetooth Wireless Technology', 'Cruise Cont...","['iPod Connector', 'MP3 Player', 'Satellite Ra...",1,1,1,1,0,1,1,1,1,1,1,0,1,1,0,0
1,1,acura,ilx,2019,"$26,448",28 MPG,201 HP,4.4,4.4,5.0,"['Bluetooth Wireless Technology', 'Cruise Cont...","['iPod Connector', 'MP3 Player', 'Bluetooth St...",1,1,1,1,0,1,1,1,1,1,1,0,1,1,0,0
2,2,acura,ilx,2018,"$25,828",29 MPG,201 HP,4.3,4.4,5.0,"['Bluetooth Wireless Technology', 'Cruise Cont...","['iPod Connector', 'MP3 Player', 'Satellite Ra...",1,1,1,1,0,1,1,1,1,1,1,0,1,1,0,0
3,3,acura,ilx,2017,"$25,296",29 MPG,201 HP,4.4,4.4,5.0,"['Bluetooth Wireless Technology', 'Cruise Cont...","['iPod Connector', 'MP3 Player', 'Premium Radi...",1,1,1,1,0,1,1,1,1,1,1,0,1,1,0,0
4,4,acura,ilx,2016,"$24,971",29 MPG,201 HP,4.4,4.4,5.0,"['Bluetooth Wireless Technology', 'Cruise Cont...","['iPod Connector', 'MP3 Player', 'Premium Radi...",1,1,1,1,0,1,1,1,1,1,1,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,368,toyota,86,2017,"$26,321",27 MPG,205 HP,4.6,5.0,NaN,"['Bluetooth Wireless Technology', 'Cruise Cont...","['iPod Connector', 'Premium Radio', 'Bluetooth...",1,1,1,1,0,1,1,1,1,1,1,0,1,1,0,0
369,369,toyota,gr-supra,2020,"$50,945",26 MPG,335 HP,0.0,4.4,NaN,"['Bluetooth Wireless Technology', 'Cruise Cont...","['iPod Connector', 'Premium Radio', 'Satellite...",1,1,1,1,0,1,1,1,1,1,1,0,1,1,0,0
370,370,toyota,corolla-hatchback,2020,"$21,245",31 MPG,168 HP,0.0,4.8,5.0,"['Bluetooth Wireless Technology', 'Cruise Cont...","['Satellite Radio', 'Bluetooth Streaming Audio']",1,1,1,1,0,1,1,1,1,1,1,0,1,1,0,0
371,371,toyota,corolla-hatchback,2019,"$20,869",31 MPG,168 HP,0.0,4.8,5.0,"['Bluetooth Wireless Technology', 'Cruise Cont...","['iPod Connector', 'MP3 Player', 'Bluetooth St...",1,1,1,1,0,1,1,1,1,1,1,0,1,1,0,0


In [0]:
Global_df

,Unnamed: 0,Maker,Model,Year,New_Price,MPG,HP,Expert-Rating,Consumer-Rating,Safety-Rating,Technology,Entertainment,Bluetooth Wireless Technology,Cruise Control,Hands Free Phone,Remote Keyless Entry,Audible Text Messaging,Proximity Sensing Keyless Entry,Push-Button Engine Start,Smartphone Interface,USB Port,Voice Recognition System,iPod Connector,MP3 Player,Satellite Radio,Bluetooth Streaming Audio,CD Player,iPod Interface
0,0,acura,ilx,2020,"$26,925",28 MPG,201 HP,4.4,4.4,5.0,"['Bluetooth Wireless Technology', 'Cruise Cont...","['iPod Connector', 'MP3 Player', 'Satellite Ra...",1,1,1,1,0,1,1,1,1,1,1,0,1,1,0,0
1,1,acura,ilx,2019,"$26,448",28 MPG,201 HP,4.4,4.4,5.0,"['Bluetooth Wireless Technology', 'Cruise Cont...","['iPod Connector', 'MP3 Player', 'Bluetooth St...",1,1,1,1,0,1,1,1,1,1,1,0,1,1,0,0
2,2,acura,ilx,2018,"$25,828",29 MPG,201 HP,4.3,4.4,5.0,"['Bluetooth Wireless Technology', 'Cruise Cont...","['iPod Connector', 'MP3 Player', 'Satellite Ra...",1,1,1,1,0,1,1,1,1,1,1,0,1,1,0,0
3,3,acura,ilx,2017,"$25,296",29 MPG,201 HP,4.4,4.4,5.0,"['Bluetooth Wireless Technology', 'Cruise Cont...","['iPod Connector', 'MP3 Player', 'Premium Radi...",1,1,1,1,0,1,1,1,1,1,1,0,1,1,0,0
4,4,acura,ilx,2016,"$24,971",29 MPG,201 HP,4.4,4.4,5.0,"['Bluetooth Wireless Technology', 'Cruise Cont...","['iPod Connector', 'MP3 Player', 'Premium Radi...",1,1,1,1,0,1,1,1,1,1,1,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,368,toyota,86,2017,"$26,321",27 MPG,205 HP,4.6,5.0,NaN,"['Bluetooth Wireless Technology', 'Cruise Cont...","['iPod Connector', 'Premium Radio', 'Bluetooth...",1,1,1,1,0,1,1,1,1,1,1,0,1,1,0,0
369,369,toyota,gr-supra,2020,"$50,945",26 MPG,335 HP,0.0,4.4,NaN,"['Bluetooth Wireless Technology', 'Cruise Cont...","['iPod Connector', 'Premium Radio', 'Satellite...",1,1,1,1,0,1,1,1,1,1,1,0,1,1,0,0
370,370,toyota,corolla-hatchback,2020,"$21,245",31 MPG,168 HP,0.0,4.8,5.0,"['Bluetooth Wireless Technology', 'Cruise Cont...","['Satellite Radio', 'Bluetooth Streaming Audio']",1,1,1,1,0,1,1,1,1,1,1,0,1,1,0,0
371,371,toyota,corolla-hatchback,2019,"$20,869",31 MPG,168 HP,0.0,4.8,5.0,"['Bluetooth Wireless Technology', 'Cruise Cont...","['iPod Connector', 'MP3 Player', 'Bluetooth St...",1,1,1,1,0,1,1,1,1,1,1,0,1,1,0,0


In [0]:
from     bs4 import  BeautifulSoup as soup
import   requests
import   pandas as pd
from     urllib.request  import  urlopen
# Bring CSV files (e.g., Maker1_df.csv) into memory as Data Frame objects (e.g., Maker1_df):
Maker1_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker1_df.csv')
Maker17_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker17_df.csv')
Maker18_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker18_df.csv')
Maker41_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker41_df.csv')
Maker42_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker42_df.csv')

# Concatenate these Data Frame files into one Data Frame
df = [ Maker1_df, Maker17_df, Maker18_df, Maker41_df, Maker42_df ] 


In [0]:
from     bs4 import  BeautifulSoup as soup
import   requests
import   pandas as pd
from     urllib.request  import  urlopen
# Bring CSV files (e.g., Maker1_df.csv) into memory as Data Frame objects (e.g., Maker1_df):
Maker1_df = pd.read_csv(r'/content/F:\Users\AdamEzzat\Springboard\Makers-DFs\Maker1_df.csv')
Maker17_df = pd.read_csv(r'/content/F:\Users\AdamEzzat\Springboard\Makers-DFs\Maker17_df.csv')
Maker18_df = pd.read_csv(r'/content/F:\Users\AdamEzzat\Springboard\Makers-DFs\Maker18_df.csv')
Maker41_df = pd.read_csv(r'/content/F:\Users\AdamEzzat\Springboard\Makers-DFs\Maker41_df.csv')
Maker42_df = pd.read_csv(r'/content/F:\Users\AdamEzzat\Springboard\Makers-DFs\Maker42_df.csv')

# Concatenate these Data Frame files into one Data Frame
df = [ Maker1_df, Maker17_df, Maker18_df, Maker41_df, Maker42_df ] 

# Store this single Data Frame file as CSV file on local disk
Global_df = pd.concat(df)

# Bring CSV files (e.g., Maker1_df.csv) into memory as Data Frame objects (e.g., Maker1_df):
Global_df.to_csv (r'\content/F:\Users\AdamEzzat\Springboard\Makers-DFs\Global_df.csv')

# Ben’s code to expand the Technology column into multiple columns in the Global_df
import ast
sample_dict = ast.literal_eval(Global_df.head()['property'].iloc[0])
print(sample_dict, "\n\n")

Global_df['property_dict']= Global_df['property'].apply(lambda x: ast.literal_eval(x))
for key in sample_dict: 
    Global_df[key] = Global_df['property_dict'].apply(lambda x: x[key] if key in x else "ERROR")
all_tech = set(Global_df['Technology'].sum())
for tech in all_tech: 
    Global_df[tech] = Global_df['Technology'].apply(lambda x: 1 if tech in x else 0)
Global_df
# Adam’s code to expand the Entertainment column into multiple columns in the Global_df
all_ent = set(Global_df['Entertainment'].sum())
for ent in all_ent: 
    Global_df[ent] = Global_df['Entertainment'].apply(lambda x: 1 if ent in x else 0)

print(Global_df)






{'MPG': '19 MPG', 'HP': '270 HP', 'Expert-Rating': '4.8', 'Consumer-Rating': '4.9', 'Safety-Rating': 'N/A', 'Technology': ['Cruise Control'], 'Entertainment': ['CD Player']}
     Unnamed: 0   Maker  ... Satellite Radio  iPod Connector
0             0   acura  ...               0               0
1             1   acura  ...               0               0
2             2   acura  ...               0               0
3             3   acura  ...               0               0
4             4   acura  ...               0               0
..          ...     ...  ...             ...             ...
368         368  toyota  ...               1               1
369         369  toyota  ...               1               1
370         370  toyota  ...               1               1
371         371  toyota  ...               1               1
372         372  toyota  ...               1               1

[963 rows x 33 columns]


In [0]:
from     bs4 import  BeautifulSoup as soup
import   requests
import   pandas as pd
from     urllib.request  import  urlopen
# Bring CSV files (e.g., Maker1_df.csv) into memory as Data Frame objects (e.g., Maker1_df):
Maker1_df = pd.read_csv(r'/content/F__Users_AdamEzzat_Springboard_Makers-DFs_Maker1_df.csv')
Maker17_df = pd.read_csv(r/content/F__Users_AdamEzzat_Springboard_Makers-DFs_Maker17_df.csv')
Maker18_df = pd.read_csv(r'/content/F:\Users\AdamEzzat\Springboard\Makers-DFs\Maker18_df.csv')
Maker41_df = pd.read_csv(r'/content/F:\Users\AdamEzzat\Springboard\Makers-DFs\Maker41_df.csv')
Maker42_df = pd.read_csv(r'/content/F:\Users\AdamEzzat\Springboard\Makers-DFs\Maker42_df.csv')

# Concatenate these Data Frame files into one Data Frame
df = [ Maker1_df, Maker17_df, Maker18_df, Maker41_df, Maker42_df ] 
# Store this single Data Frame file as CSV file on local disk
Global_df = pd.concat(df)


FileNotFoundError: ignored

In [0]:
set(Global_df.year.unique()).intersection(set(Pakistan_df.year.unique())/(set(Global_df.year.unique()) + set(Pakistan_df_.unique()))

SyntaxError: ignored

In [0]:
Pakistan_df = pd.read_csv('Pakistan_Data.csv' , encoding='cp1252')

In [0]:
import ast
sample_dict = ast.literal_eval(Global_df.head()['property'].iloc[0])
print(sample_dict)

{'MPG': '19 MPG', 'HP': '270 HP', 'Expert-Rating': '4.8', 'Consumer-Rating': '4.9', 'Safety-Rating': 'N/A', 'Technology': ['Cruise Control'], 'Entertainment': ['CD Player']}


In [0]:
Global_df['property_dict']= Global_df['property'].apply(lambda x: ast.literal_eval(x))

In [0]:
for key in sample_dict: 
    Global_df[key] = Global_df['property_dict'].apply(lambda x: x[key] if key in x else "ERROR")

In [0]:
all_tech = set(Global_df['Technology'].sum())

In [0]:
for tech in all_tech: 
    Global_df[tech] = Global_df['Technology'].apply(lambda x: 1 if tech in x else 0)

In [0]:
Global_df

,Unnamed: 0,Maker,Model,Year,New_Price,property,property_dict,MPG,HP,Expert-Rating,Consumer-Rating,Safety-Rating,Technology,Entertainment,Hands Free Phone,USB Port,Bluetooth Wireless Technology,Push-Button Engine Start,Navigation System,Smartphone Interface,Internet Access,Voice Recognition System,Cruise Control,Proximity Sensing Keyless Entry,Touch Screen Monitor,Real-Time Traffic Information,Remote Keyless Entry
0,0,acura,nsx,2019,"$159,300","{'MPG': '19 MPG', 'HP': '270 HP', 'Expert-Rati...","{'MPG': '19 MPG', 'HP': '270 HP', 'Expert-Rati...",19 MPG,270 HP,4.8,4.9,N/A,[Cruise Control],[CD Player],0,0,0,0,0,0,0,0,1,0,0,0,0
1,1,acura,rdx,2020,"$38,825","{'MPG': '19 MPG', 'HP': '270 HP', 'Expert-Rati...","{'MPG': '19 MPG', 'HP': '270 HP', 'Expert-Rati...",19 MPG,270 HP,4.8,4.9,N/A,[Cruise Control],[CD Player],0,0,0,0,0,0,0,0,1,0,0,0,0
2,2,acura,mdx-sport-hybrid,2020,"$54,025","{'MPG': '19 MPG', 'HP': '270 HP', 'Expert-Rati...","{'MPG': '19 MPG', 'HP': '270 HP', 'Expert-Rati...",19 MPG,270 HP,4.8,4.9,N/A,[Cruise Control],[CD Player],0,0,0,0,0,0,0,0,1,0,0,0,0
3,3,acura,mdx,2020,"$45,525","{'MPG': '19 MPG', 'HP': '270 HP', 'Expert-Rati...","{'MPG': '19 MPG', 'HP': '270 HP', 'Expert-Rati...",19 MPG,270 HP,4.8,4.9,N/A,[Cruise Control],[CD Player],0,0,0,0,0,0,0,0,1,0,0,0,0
4,4,acura,tlx,2020,"$34,025","{'MPG': '19 MPG', 'HP': '270 HP', 'Expert-Rati...","{'MPG': '19 MPG', 'HP': '270 HP', 'Expert-Rati...",19 MPG,270 HP,4.8,4.9,N/A,[Cruise Control],[CD Player],0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,368,toyota,prius-c,2012,"$20,142","{'MPG': '35 MPG', 'HP': '106 HP', 'Expert-Rati...","{'MPG': '35 MPG', 'HP': '106 HP', 'Expert-Rati...",35 MPG,106 HP,0.0,5,5.0,"[Bluetooth Wireless Technology, Cruise Control...","[iPod Connector, Satellite Radio, Bluetooth St...",1,1,1,1,1,1,1,1,1,1,1,0,1
369,369,toyota,86,2019,"$27,519","{'MPG': '35 MPG', 'HP': '106 HP', 'Expert-Rati...","{'MPG': '35 MPG', 'HP': '106 HP', 'Expert-Rati...",35 MPG,106 HP,0.0,5,5.0,"[Bluetooth Wireless Technology, Cruise Control...","[iPod Connector, Satellite Radio, Bluetooth St...",1,1,1,1,1,1,1,1,1,1,1,0,1
370,370,toyota,86,2018,"$26,874","{'MPG': '35 MPG', 'HP': '106 HP', 'Expert-Rati...","{'MPG': '35 MPG', 'HP': '106 HP', 'Expert-Rati...",35 MPG,106 HP,0.0,5,5.0,"[Bluetooth Wireless Technology, Cruise Control...","[iPod Connector, Satellite Radio, Bluetooth St...",1,1,1,1,1,1,1,1,1,1,1,0,1
371,371,toyota,86,2017,"$26,321","{'MPG': '35 MPG', 'HP': '106 HP', 'Expert-Rati...","{'MPG': '35 MPG', 'HP': '106 HP', 'Expert-Rati...",35 MPG,106 HP,0.0,5,5.0,"[Bluetooth Wireless Technology, Cruise Control...","[iPod Connector, Satellite Radio, Bluetooth St...",1,1,1,1,1,1,1,1,1,1,1,0,1


In [0]:
all_ent = set(Global_df['Entertainment'].sum())

In [0]:
for ent in all_ent: 
    Global_df[ent] = Global_df['Entertainment'].apply(lambda x: 1 if ent in x else 0)

In [0]:
print (Global_df)

     Unnamed: 0   Maker  ... iPod Connector  Bluetooth Streaming Audio
0             0   acura  ...              0                          0
1             1   acura  ...              0                          0
2             2   acura  ...              0                          0
3             3   acura  ...              0                          0
4             4   acura  ...              0                          0
..          ...     ...  ...            ...                        ...
368         368  toyota  ...              1                          1
369         369  toyota  ...              1                          1
370         370  toyota  ...              1                          1
371         371  toyota  ...              1                          1
372         372  toyota  ...              1                          1

[963 rows x 33 columns]


In [0]:
Global_df['Year'] = Global_df.Year.astype (float)

In [0]:
Global_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 963 entries, 0 to 372
Data columns (total 33 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Unnamed: 0                       963 non-null    int64  
 1   Maker                            963 non-null    object 
 2   Model                            963 non-null    object 
 3   Year                             963 non-null    float64
 4   New_Price                        963 non-null    object 
 5   property                         963 non-null    object 
 6   property_dict                    963 non-null    object 
 7   MPG                              963 non-null    object 
 8   HP                               963 non-null    object 
 9   Expert-Rating                    963 non-null    object 
 10  Consumer-Rating                  963 non-null    object 
 11  Safety-Rating                    963 non-null    object 
 12  Technology            

In [0]:
len(set(Global_df.Year.unique()).intersection(set(Pakistan_df.Year.unique())))/len(set(Global_df.Year.unique()) + set(Pakistan_df.unique()))

AttributeError: ignored

In [0]:
KBB_df = Pakistan_df.merge (Global_df, left_on = ['Brand', 'Model','Year'], right_on = ['Maker', 'Model', 'Year'], how = 'inner')

In [0]:
KBB_df

,Brand,Condition,Fuel,KMs Driven,Model,Price,Registered City,Transaction Type,Year,Unnamed: 0,Maker,New_Price,property,property_dict,MPG,HP,Expert-Rating,Consumer-Rating,Safety-Rating,Technology,Entertainment,Hands Free Phone,USB Port,Bluetooth Wireless Technology,Push-Button Engine Start,Navigation System,Smartphone Interface,Internet Access,Voice Recognition System,Cruise Control,Proximity Sensing Keyless Entry,Touch Screen Monitor,Real-Time Traffic Information,Remote Keyless Entry,Satellite Radio,MP3 Player,iPod Connector,CD Player,Bluetooth Streaming Audio,Premium Radio


In [0]:
KBB_df.head()

,Brand,Condition,Fuel,KMs Driven,Model,Price,Registered City,Transaction Type,Year,Unnamed: 0,Maker,New_Price,property,property_dict,MPG,HP,Expert-Rating,Consumer-Rating,Safety-Rating,Technology,Entertainment,Cruise Control,Hands Free Phone,Voice Recognition System,USB Port,Proximity Sensing Keyless Entry,Navigation System,Remote Keyless Entry,Internet Access,Touch Screen Monitor,Push-Button Engine Start,Bluetooth Wireless Technology,Smartphone Interface,Real-Time Traffic Information,CD Player,Satellite Radio,MP3 Player,Premium Radio,iPod Connector,Bluetooth Streaming Audio


In [0]:
print(KBB_df)

Empty DataFrame
Columns: [Brand, Condition, Fuel, KMs Driven, Model, Price, Registered City, Transaction Type, Year, Unnamed: 0, Maker, New_Price, property, property_dict, MPG, HP, Expert-Rating, Consumer-Rating, Safety-Rating, Technology, Entertainment, Cruise Control, Hands Free Phone, Voice Recognition System, USB Port, Proximity Sensing Keyless Entry, Navigation System, Remote Keyless Entry, Internet Access, Touch Screen Monitor, Push-Button Engine Start, Bluetooth Wireless Technology, Smartphone Interface, Real-Time Traffic Information, CD Player, Satellite Radio, MP3 Player, Premium Radio, iPod Connector, Bluetooth Streaming Audio]
Index: []


In [0]:
Global_df.merge(Pakistan_df, left_on = ['Maker', 'Model','Year'], right_on = ['Brand', 'Model','Year'], how = 'inner')

,Unnamed: 0,Unnamed: 0.1,Maker,Model,Year,New_Price,property,Brand,Condition,Fuel,KMs Driven,Price,Registered City,Transaction Type
